# Intro
This kernel has used contributes/snippets from many others:

* Forked from https://www.kaggle.com/prashantkikani/help-humanity-by-helping-robots
* Feature Engineering from https://www.kaggle.com/jesucristo/1-smart-robots-complete-notebook-0-73
* Group_id insight from https://www.kaggle.com/c/career-con-2019/discussion/87239#latest-507715
* Additional Features from https://www.kaggle.com/taigokuriyama/random-forest-baseline
* ... (other cites are in the code, I apologise if I forgot someone)

As for me, I tried some solutions which, however, didn't work as expected. For example:

* XGB (the validation error plot is fantastic, but the LB doesn't agree)
* Groups/No Groups Training of RF/XGB Classifier (ok, a better esteem of validation error, and then?)
* Group Prediction (with a simple SVM, the accuracy is similar to the one of leakage, but...)
* PCA (Useless, but the plots are nice)
* ...

As a submission, I finally used the output of the basic classifier ("RF No Groups") which scored 0.71 in the public LB and 0.61 in the private one.

Probably my best error has been relying too much on the "distribution hack" discussion. Until the end, I expected that the real distribution had few surfaces with id=7,8.

Anyway, a great experience. Thanks to Kaggle and congratulations to the Winners!

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
from seaborn import countplot,lineplot, barplot
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
import scipy as sp
from functools import partial
from collections import Counter

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.svm import SVC

In [ ]:
tr = pd.read_csv('../input/X_train.csv')
te = pd.read_csv('../input/X_test.csv')
target = pd.read_csv('../input/y_train.csv')
ss = pd.read_csv('../input/sample_submission.csv')

In [ ]:
tr0=tr
te0=te

Now one important thing that I initially haven't noted.

In [ ]:
print(tr.shape[0])
print(target.shape[0])

In [ ]:
print(te.shape[0])
print(ss.shape[0])

In [ ]:
df_target=pd.read_csv('../input/y_train.csv')
df_target.head()

The label to predict is related to series_id. The measurement_id it's a detail present in the dataset called train and test but is actually we have to make our predictions on ss learning from target. In other words measurement_id is a detail that we have to delete (by grouping).

In [ ]:
le=LabelEncoder()
df_target['surface_id']=le.fit_transform(df_target['surface'])
df_target.head()

In [ ]:
df_target_s=df_target[['surface','surface_id']].groupby(by='surface').max().reset_index()
df_target_s

In [ ]:
df_target_cnt=target.groupby(by='surface').count()['series_id']
df_target_cnt=df_target_cnt.reset_index()

In [ ]:
df_target_s=df_target_s.merge(df_target_cnt,on='surface')
df_target_s

In [ ]:
barplot(x='surface_id',y='series_id',data=df_target_s)
plt.show()

In [ ]:
# https://www.kaggle.com/jesucristo/1-smart-robots-complete-notebook-0-73/notebook
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(tr.iloc[:,3:].corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

What is a group? From the data description: "ID number for all of the measurements taken in a recording session. Provided for the training set only, to enable more cross validation strategies." We'll merge it to the training set. And for the test set we'll invent something ...

## Groups ...?

OK, now we can study a little better what is a group. Let's take in consideration for example, this one:

In [ ]:
df_target[df_target['group_id']==5]

In [ ]:
df_target_g=df_target[['group_id','surface_id']].groupby(by='group_id').count().reset_index()
df_target_g.head()

In [ ]:
yg=df_target['group_id'].copy()
fig,ax = plt.subplots(1,1,figsize=(18,4))
countplot(yg, ax=ax)

In [ ]:
yg.value_counts(ascending=True).head()

In [ ]:
yg.value_counts(ascending=True).head()

In [ ]:
print('Groups Nr Max Min',yg.nunique(),yg.max(),yg.min())

## Feature Engineering

In [ ]:
target['surface'] = le.fit_transform(target['surface'])

In [ ]:
# https://stackoverflow.com/questions/53033620/how-to-convert-euler-angles-to-quaternions-and-get-the-same-euler-angles-back-fr?rq=1
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

In [ ]:
def fe_step0 (actual):
    
    # https://www.mathworks.com/help/aeroblks/quaternionnorm.html
    # https://www.mathworks.com/help/aeroblks/quaternionmodulus.html
    # https://www.mathworks.com/help/aeroblks/quaternionnormalize.html
        
    actual['norm_quat'] = (actual['orientation_X']**2 + actual['orientation_Y']**2 + actual['orientation_Z']**2 + actual['orientation_W']**2)
    actual['mod_quat'] = (actual['norm_quat'])**0.5
    actual['norm_X'] = actual['orientation_X'] / actual['mod_quat']
    actual['norm_Y'] = actual['orientation_Y'] / actual['mod_quat']
    actual['norm_Z'] = actual['orientation_Z'] / actual['mod_quat']
    actual['norm_W'] = actual['orientation_W'] / actual['mod_quat']
    
    return actual

In [ ]:
tr = fe_step0(tr)
te = fe_step0(te)

In [ ]:
def fe_step1 (actual):
    """Quaternions to Euler Angles"""
    
    x, y, z, w = actual['norm_X'].tolist(), actual['norm_Y'].tolist(), actual['norm_Z'].tolist(), actual['norm_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    actual['euler_x'] = nx
    actual['euler_y'] = ny
    actual['euler_z'] = nz
    return actual

In [ ]:
def feat_eng(data):
    
    data['totl_anglr_vel'] = (data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 + data['angular_velocity_Z']**2)** 0.5
    data['totl_linr_acc'] = (data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 + data['linear_acceleration_Z']**2)**0.5
    data['totl_xyz'] = (data['orientation_X']**2 + data['orientation_Y']**2 + data['orientation_Z']**2)**0.5
    data['totl_angle'] = np.sqrt(data['euler_x'] ** 2 + data['euler_y'] ** 2 + data['euler_z'] ** 2)
    
    # absolute values
    data['linear_acceleration_X_abs'] = data['linear_acceleration_X'].where(data['linear_acceleration_X']>=0, - data['linear_acceleration_X'])
    data['linear_acceleration_Y_abs'] = data['linear_acceleration_Y'].where(data['linear_acceleration_Y']>=0, - data['linear_acceleration_Y'])
    data['linear_acceleration_Z_abs'] = data['linear_acceleration_Z'].where(data['linear_acceleration_Z']>=0, - data['linear_acceleration_Z'])

    # vs features
    data['acc_vs_vel'] = data['totl_linr_acc'] / data['totl_anglr_vel']
    data['angle_vs_acc'] = data['totl_angle'] / data['totl_linr_acc']
    data['angle_vs_vel'] = data['totl_angle'] / data['totl_anglr_vel']
    
    # interaction features
    data['acc_int_vel'] = data['totl_linr_acc'] * data['totl_anglr_vel']
    data['angle_int_acc'] = data['totl_angle'] * data['totl_linr_acc']
    data['angle_int_vel'] = data['totl_angle'] * data['totl_anglr_vel']
    data['angle_ints_vel_acc'] = data['totl_angle'] * data['totl_anglr_vel'] * data['totl_linr_acc']
    
    def mean_change_of_abs_change(x):
        return np.mean(np.diff(np.abs(np.diff(x))))
    
    df = pd.DataFrame() 
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number']:
            continue
        df[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        df[col + '_median'] = data.groupby(['series_id'])[col].median()
        df[col + '_max'] = data.groupby(['series_id'])[col].max()
        df[col + '_min'] = data.groupby(['series_id'])[col].min()
        df[col + '_std'] = data.groupby(['series_id'])[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(['series_id'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_mean_change_of_abs_change'] = data.groupby('series_id')[col].apply(mean_change_of_abs_change)
        df[col + '_abs_max'] = data.groupby(['series_id'])[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(['series_id'])[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
    return df

In [ ]:
tr = fe_step1(tr)
te = fe_step1(te)

In [ ]:
%%time
tr = feat_eng(tr)
te = feat_eng(te)
tr.head()

At this point we trashed measurement_id

In [ ]:
print(tr.shape)
print(te.shape)

## Scaling & Fillna

In [ ]:
#https://www.kaggle.com/donkeys/my-little-eda-with-random-forest
#function to scale two dataframes. fit and transform the first (e.g., training set), 
#and use the same scaler to transform the seconds one (e.g., test set)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
def scale_df(df1, df2, feature_cols):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df1[feature_cols])
    df1[feature_cols] = scaled_features
    scaled_test_features = scaler.transform(df2[feature_cols])
    df2[feature_cols] = scaled_test_features
    return df1, df2

In [ ]:
tr,te=scale_df(tr,te,te.columns)

In [ ]:
tr.fillna(0, inplace = True)
te.fillna(0, inplace = True)

In [ ]:
tr.replace(-np.inf, 0, inplace = True)
tr.replace(np.inf, 0, inplace = True)
te.replace(-np.inf, 0, inplace = True)
te.replace(np.inf, 0, inplace = True)

In [ ]:
print(tr.shape,te.shape)

## Group Guessing

In [ ]:
tr.columns

In [ ]:
svc_cols=[x for x in tr.columns if 'orientation_W' in x]
svc_cols=svc_cols+[x for x in tr.columns if 'orientation_Z' in x]

In [ ]:
clf_cols=[x for x in tr.columns if not (x in svc_cols)]+['group_id']

In [ ]:
X=tr.join(df_target['group_id'],on='series_id')

In [ ]:
y=X['group_id']
X=X[svc_cols]
T=te[svc_cols]

In [ ]:
print(X.shape,T.shape)

In [ ]:
CV_STEPS=10
folds = StratifiedKFold(n_splits=CV_STEPS, shuffle=True, random_state=876)

In [ ]:
#
#   SVC Groups Prediction
#
i=0
tst_preds_gr = np.zeros((T.shape[0], CV_STEPS))
oof_preds_gr = np.zeros((X.shape[0]))
splitter=folds.split(X,y)
for (train_index, valid_index) in (splitter):
    print('='*20, i, '='*20)
    X_train = X.iloc[train_index]
    print('train size:',len(train_index))
    X_valid = X.iloc[valid_index]
    print('valid size:',len(valid_index))
    y_train = y.iloc[train_index]
    y_valid = y.iloc[valid_index]
    clf =  SVC(kernel='linear')
    clf.fit(X_train, y_train)
    oof_preds_gr[valid_index] = clf.predict(X_valid)
    tst_preds_gr[:,i] = clf.predict(T)
    print('score ', accuracy_score(oof_preds_gr[valid_index], y_valid.values))
    i=i+1

In [ ]:
def extr_test_pred(tst_preds_cv):
    gr_preds=np.zeros(len(tst_preds_cv),dtype=int)
    for r in range(0,len(tst_preds_cv)):
        gr_preds[r]=Counter(tst_preds_cv[r,:]).most_common(1)[0][0]
    return gr_preds

In [ ]:
tr['group_id']=y

In [ ]:
te['group_id']=extr_test_pred(tst_preds_gr)

In [ ]:
print(tr.shape,te.shape)

## Final Data

In [ ]:
X=tr[clf_cols]
T=te[clf_cols]
y=pd.DataFrame(target['surface'],index=tr.index)
LABELS=[0,1,2,3,4,5,6,7,8]

In [ ]:
X.to_csv('X.csv', index=False)
T.to_csv('T.csv', index=False)
y.to_csv('y.csv', index=False)

## Random Forest

In [ ]:
CV_STEPS=30
folds = StratifiedKFold(n_splits=CV_STEPS, shuffle=True, random_state=1984)

In [ ]:
groups=yg.unique()
dummy=np.zeros(len(groups))

In [ ]:
def group_id_drop(W):
    try:
        W.drop('group_id',axis=1,inplace=True)
    except:
        print('Info: group_id already dropped.')
    return(W)

In [ ]:
#
#   Random Forest Groups Based
#
tst_preds_rf = np.zeros((T.shape[0], CV_STEPS))
oof_preds_rf = np.zeros((X.shape[0]))
importances = np.zeros((CV_STEPS,X.shape[1]-1)) # group_id will be deleted
i=0
splitter=folds.split(groups,dummy)
for (tr_gr_idx, va_gr_idx) in (splitter):
    print('-'*20, i, '-'*20)
    tr_groups=groups[tr_gr_idx]
    va_groups=groups[va_gr_idx]
    X_train = X[X['group_id'].isin(tr_groups)]
    X_train.drop('group_id',axis=1,inplace=True)
    train_index = X_train.index
    print('train size:',len(train_index))
    X_valid = X[X['group_id'].isin(va_groups)]
    X_valid.drop('group_id',axis=1,inplace=True)
    valid_index = X_valid.index
    print('valid size:',len(valid_index))
    y_train = y.iloc[train_index]
    curr_labels=y_train['surface'].unique()
    print('used labels:',len(curr_labels),' [',','.join([str(c) for c in curr_labels]),']')
    y_valid = y.iloc[valid_index]
    clf =  RandomForestClassifier(n_estimators = 500, n_jobs = -1)
    clf.fit(X_train, y_train)
    oof_preds_rf[valid_index] = clf.predict(X_valid)
    TWG=T.drop('group_id',axis=1)
    tst_preds_rf[:,i] = clf.predict(TWG)
    ass_labels=np.unique(tst_preds_rf[:,i])
    print('assigned labels:',len(ass_labels),' [',','.join([str(int(c)) for c in ass_labels]),']')
    print('score ', accuracy_score(oof_preds_rf[valid_index], y_valid.values))
    importances[i,0:X.shape[1]] = clf.feature_importances_
    i=i+1

In [ ]:
def plot_hists(oof,sub,title_oof,title_sub):
    fig,ax=plt.subplots(1,3,figsize=(12,4))
    countplot(target['surface'], ax=ax[0], order=LABELS).set_title('Train (Labels)')
    countplot(oof, ax=ax[1],order=LABELS).set_title(title_oof)
    countplot(sub, ax=ax[2],order=LABELS).set_title(title_sub)

In [ ]:
# https://www.kaggle.com/artgor/where-do-the-robots-drive
import itertools

def plot_confusion_matrix(truth, pred, classes, normalize=False, title=''):
    cm = confusion_matrix(truth, pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix', size=15)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.tight_layout()

In [ ]:
tst_preds_rf0 = tst_preds_rf
oof_preds_rf0 = oof_preds_rf
importances0 = importances

In [ ]:
preds_rf0=extr_test_pred(tst_preds_rf0)

In [ ]:
plot_confusion_matrix(target['surface'], oof_preds_rf0, le.classes_)

In [ ]:
plot_hists(oof_preds_rf0,preds_rf0,'OOF Train (RF Groups)','Test (RF Groups)')

In [ ]:
accuracy_score(target['surface'], oof_preds_rf0)

In [ ]:
#
#   Random Forest No Groups
#
i=0
tst_preds_rf = np.zeros((T.shape[0], CV_STEPS))
oof_preds_rf = np.zeros((X.shape[0]))
importances = np.zeros((CV_STEPS,X.shape[1]-1)) # group_id will be deleted
splitter=folds.split(X,y)
for (train_index, valid_index) in (splitter):
    print('#'*20, i, '#'*20)
    X_train = X.iloc[train_index]
    print('train size:',len(train_index))
    X_valid = X.iloc[valid_index]
    print('valid size:',len(valid_index))
    y_train = y.iloc[train_index]
    curr_labels=y_train['surface'].unique()
    X_train=group_id_drop(X_train)
    X_valid=group_id_drop(X_valid)
    print('used labels:',len(curr_labels),' [',','.join([str(c) for c in curr_labels]),']')
    y_valid = y.iloc[valid_index]
    clf =  RandomForestClassifier(n_estimators = 500, n_jobs = -1)
    clf.fit(X_train, y_train)
    oof_preds_rf[valid_index] = clf.predict(X_valid)
    TWG=T.drop('group_id',axis=1)
    tst_preds_rf[:,i] = clf.predict(TWG)
    ass_labels=np.unique(tst_preds_rf[:,i])
    print('assigned labels:',len(ass_labels),' [',','.join([str(int(c)) for c in ass_labels]),']')
    print('score ', accuracy_score(oof_preds_rf[valid_index], y_valid.values))
    importances[i,0:X.shape[1]] = clf.feature_importances_
    i=i+1

In [ ]:
preds_rf=extr_test_pred(tst_preds_rf)

In [ ]:
tst_preds_rf[0,:]

In [ ]:
preds_rf[0]

In [ ]:
dfi=pd.DataFrame(index=X_train.columns)
dfi['importance']=importances.mean(axis=0)
dfi=dfi.sort_values(by='importance',ascending=False)
dfi.head()

In [ ]:
dfi=dfi.head(20)
fig,ax=plt.subplots(1,1,figsize=(18,10))
barplot(y=dfi.index,x=dfi['importance'],ax=ax)

In [ ]:
plot_confusion_matrix(target['surface'], oof_preds_rf, le.classes_)

In [ ]:
plot_hists(oof_preds_rf,preds_rf,'OOF Train (RF Classifier)','Test (RF Classifier)')

In [ ]:
accuracy_score(target['surface'], oof_preds_rf)

## XGBoost

In [ ]:
# It doesn't work, but I want to keep this section anyway
CV_STEPS=10
folds = StratifiedKFold(n_splits=CV_STEPS, shuffle=True, random_state=1984)

In [ ]:
NR_EPOCHS=1000
xgb_params = {
    'booster' : 'gbtree',
    'objective' : 'multi:softprob',
    'num_class' : 9,
    'seed': 1984,
    'eta': 0.01,
    'max_depth': 3,
    'gamma': 0.1,
    'lambda': 0.8,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'gpu_hist',
    'device': 'gpu',
    'silent': 1,
}
verbose_eval = 100
early_stop = 200

In [ ]:
def xgb_train_no_groups(params, X, y, T):
    oof_train_preds = np.zeros(X.shape[0])
    tst_preds = np.zeros((T.shape[0],CV_STEPS))
    perf_dict_list=[]
    i=0
    splitter=folds.split(X,y)
    for (train_index, valid_index) in (splitter):
        print('#'*20, i, '#'*20)
        X_train = X.iloc[train_index]
        print('train size:',len(train_index))
        X_valid = X.iloc[valid_index]
        print('valid size:',len(valid_index))
        y_train = y.iloc[train_index]
        curr_labels=y_train['surface'].unique()
        X_train=group_id_drop(X_train)
        X_valid=group_id_drop(X_valid)
        print('used labels:',len(curr_labels),' [',','.join([str(c) for c in curr_labels]),']')
        y_valid = y.iloc[valid_index]

        d_train = xgb.DMatrix(data=X_train, label=y_train, feature_names=X_train.columns)
        d_valid = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X_valid.columns)

        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        perf=dict()
        model = xgb.train(dtrain=d_train, num_boost_round=NR_EPOCHS, evals=watchlist,
                         early_stopping_rounds=early_stop, verbose_eval=verbose_eval, params=params,
                         evals_result=perf)

        valid_pred = model.predict(xgb.DMatrix(X_valid, feature_names=X_valid.columns), ntree_limit=model.best_ntree_limit)
        TWG=T.drop('group_id',axis=1)
        test_pred = model.predict(xgb.DMatrix(TWG, feature_names=TWG.columns), ntree_limit=model.best_ntree_limit)

        oof_train_preds[valid_index] = valid_pred.argmax(axis=1)
        tst_preds[:,i] = test_pred.argmax(axis=1)
        perf_dict_list = perf_dict_list + [perf]
        
        print('Accuracy =',accuracy_score(target['surface'][valid_index], valid_pred.argmax(axis=1)))

        i += 1
        
    return model, oof_train_preds, tst_preds, perf_dict_list

In [ ]:
def xgb_train_groups_based(params, X, y, T):
    oof_train_preds = np.zeros(X.shape[0])
    tst_preds = np.zeros((T.shape[0],CV_STEPS))
    perf_dict_list=[]
    i=0
    splitter=folds.split(groups,dummy)
    for (tr_gr_idx, va_gr_idx) in (splitter):
        print('-'*20, i, '-'*20)
        tr_groups=groups[tr_gr_idx]
        va_groups=groups[va_gr_idx]
        X_train = X[X['group_id'].isin(tr_groups)]
        X_train.drop('group_id',axis=1,inplace=True)
        train_index = X_train.index
        print('train size:',len(train_index))
        X_valid = X[X['group_id'].isin(va_groups)]
        X_valid.drop('group_id',axis=1,inplace=True)
        valid_index = X_valid.index
        print('valid size:',len(valid_index))
        y_train = y.iloc[train_index]
        curr_labels=y_train['surface'].unique()
        print('used labels:',len(curr_labels))
        y_valid = y.iloc[valid_index]

        d_train = xgb.DMatrix(data=X_train, label=y_train, feature_names=X_train.columns)
        d_valid = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X_valid.columns)

        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        perf=dict()
        model = xgb.train(dtrain=d_train, num_boost_round=NR_EPOCHS, evals=watchlist,
                         early_stopping_rounds=early_stop, verbose_eval=verbose_eval, params=params,
                         evals_result=perf)

        valid_pred = model.predict(xgb.DMatrix(X_valid, feature_names=X_valid.columns), ntree_limit=model.best_ntree_limit)
        TWG=T.drop('group_id',axis=1)
        test_pred = model.predict(xgb.DMatrix(TWG, feature_names=TWG.columns), ntree_limit=model.best_ntree_limit)

        oof_train_preds[valid_index] = valid_pred.argmax(axis=1)
        tst_preds[:,i] = test_pred.argmax(axis=1)
        perf_dict_list = perf_dict_list + [perf]
        
        print('Accuracy =',accuracy_score(target['surface'][valid_index], valid_pred.argmax(axis=1)))

        i += 1
        
    return model, oof_train_preds, tst_preds, perf_dict_list

In [ ]:
%%time
model0, oof_preds_xgb0, tst_preds_xgb0, perf = xgb_train_groups_based(xgb_params, X,y,T)

In [ ]:
preds_xgb0=extr_test_pred(tst_preds_xgb0)

In [ ]:
plot_hists(oof_preds_xgb0,preds_xgb0,'OOF Train (XGB Groups)','Test (XGB Groups)')

In [ ]:
accuracy_score(target['surface'], oof_preds_xgb0)

In [ ]:
%%time
model, oof_preds_xgb, tst_preds_xgb, perf = xgb_train_no_groups(xgb_params, X,y,T)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,48))
xgb.plot_importance(model,ax=ax)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))
for i in range(0,CV_STEPS):
    ax.plot(perf[i]['train']['merror'], color='blue')
    ax.plot(perf[i]['valid']['merror'], color='red')

In [ ]:
preds_xgb=extr_test_pred(tst_preds_xgb)
preds_xgb

In [ ]:
oof_preds_xgb

In [ ]:
plot_hists(oof_preds_xgb,preds_xgb,'OOF Train (XGB)','Test (XGB)')

In [ ]:
accuracy_score(target['surface'], oof_preds_xgb)

## Prediction Choice and Analysis

In [ ]:
preds=preds_rf0
# preds=preds_rf
# preds=preds_xgb

In [ ]:
preds

In [ ]:
ss['surface'] = le.inverse_transform(preds)
ss['surface_id'] = preds
ss.head()

In [ ]:
countplot(ss['surface_id'])

In [ ]:
# Attended Distribution in Test Set (https://www.kaggle.com/donkeys/distribution-hack)
attended_prc=[0.06,0.16,0.09,0.06,0.10,0.17,0.23,0.03,0.06]
attended_nr=[p * len(ss) for p in attended_prc]
barplot(y=attended_nr,x=LABELS)

Lots of 7,8,1 (in the first plot) have to become 6 and 3. At first, is it possible to understand what are the main difference between these confused features? For this purpose we will use groups (real or predicted).

In [ ]:
XC=X.copy()
TC=T.copy()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3, svd_solver='full',random_state=1984)
pca.fit(XC)
xpc= pca.transform(XC)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
XPC = pd.DataFrame(data = xpc , columns = ['PC1', 'PC2','PC3'])
XPC['surface_id'] = y
XPC.head()

In [ ]:
def pca_plot_2d(XF):
    sns.lmplot( x="PC1", y="PC2", data=XF, fit_reg=False, hue='surface_id', legend=True,
               scatter_kws={"s": 80}, height=10, aspect=2) 

In [ ]:
selected_sufaces=[1,3,5,6,7,8]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def pca_plot_3d(XF):
    fig = plt.figure(figsize=(10,10))
    ax = Axes3D(fig)
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    ax.scatter(XF['PC1'],XF['PC2'],XF['PC3'],c=XF['surface_id'],cmap='coolwarm')

In [ ]:
XF=XPC[XPC['surface_id'].isin(selected_sufaces)]
pca_plot_2d(XF)

Group_id is practically perfect to predict the output: one group_id = one surface type (and one PC1 value). Below a 3D plot, nice but (at least for me), useless.

In [ ]:
pca_plot_3d(XF)

Now, we haven't the group_id for the test set. At first we can try to segment test set (with the predicted group) using the same components of the train set. Here is what we got:

In [ ]:
pcat=pca
#pcat = PCA(n_components=3, svd_solver='full',random_state=1984)
#pcat.fit(TC)
tpc=pcat.transform(TC)

In [ ]:
TPC = pd.DataFrame(data = tpc , columns = ['PC1','PC2','PC3'])
TPC['surface_id'] = preds
TPC.head()

In [ ]:
TF=TPC[TPC['surface_id'].isin(selected_sufaces)]
pca_plot_2d(TF)

OK, the segmentation has some problems also because the group_id has accuracy=0.5.

In [ ]:
pca_plot_3d(TF)

Let's check the coherence between predicted series_id and group_id.

In [ ]:
coh = pd.DataFrame()
coh['prev_surface_id']=preds
coh['prev_group_id']=TC['group_id']
coh['prev_id']=coh.index
coh.head()

In [ ]:
gs=df_target[['group_id','surface_id']].groupby(by=['group_id','surface_id']).first().reset_index()
gs[gs['surface_id']==1]

In [ ]:
prev_merge=coh.merge(gs,left_on=['prev_surface_id','prev_group_id'],right_on=['surface_id','group_id'],how='left')
prev_merge=prev_merge.groupby('prev_id').first()
prev_merge.fillna(-1, inplace=True)
prev_merge.head()

In [ ]:
%%time
for i in range(0,len(prev_merge)):
    gr=prev_merge['group_id'].loc[i]
    prev_gr=prev_merge['prev_group_id'].loc[i]
    if (gr==-1):
        prev_merge['surface_id'].loc[i]=gs['surface_id'][gs['group_id']==prev_gr].values[0]

In [ ]:
prev_merge=prev_merge.astype(int)
prev_merge.head()

In [ ]:
countplot(prev_merge['surface_id'])

In [ ]:
prev_merge['final_surface_id']=prev_merge['prev_surface_id']
prev_merge.head()

In [ ]:
def counter_distance(cnt,ref):
    vals=cnt.values()
    tot=sum(vals)
    cnt=sorted(cnt.items())
    dist=0
    for i in range(0,len(cnt)):
        dist+=np.abs(ref[i]-cnt[i][1]/tot)
    return dist

In [ ]:
d_freq=[]
corrections=0
for i in range(0,len(prev_merge)):
    gr=prev_merge['group_id'].loc[i]
    if (gr==-1):
        hp1=prev_merge['final_surface_id']
        hp2=hp1.copy()
        hp2[i]=prev_merge['surface_id'].loc[i]
        d_hp1=counter_distance(Counter(hp1),attended_prc)
        d_hp2=counter_distance(Counter(hp2),attended_prc)
        if (d_hp2<d_hp1):
            d_freq=d_freq+[d_hp2]
            #print('Surface subst from={h1} to from={h2}'.format(h1=d_hp1,h2=d_hp2))
            corrections=corrections+1
            prev_merge['final_surface_id'] = hp2
        else:
            d_freq=d_freq+[d_hp1]
print('Corrections made=',corrections)

In [ ]:
sns.lineplot(y=d_freq,x=range(0,len(d_freq)))

In [ ]:
prev_merge['final_surface_id'].head()

In [ ]:
Counter(prev_merge['prev_surface_id'])

In [ ]:
Counter(prev_merge['final_surface_id'])

In [ ]:
countplot(prev_merge['prev_surface_id'])

In [ ]:
countplot(prev_merge['final_surface_id'])

## Actual Submission

This is the prediction come out from classifier:

In [ ]:
ss.head()

This is the correction (comment/uncomment to apply/unapply):

In [ ]:
#ss['surface'] = le.inverse_transform(prev_merge['final_surface_id'])
#ss['surface_id'] = prev_merge['final_surface_id']
ss.head()

In [ ]:
ss.drop('surface_id',axis=1,inplace=True)
ss.to_csv('submission.csv', index=False)